<a href="https://colab.research.google.com/github/tolossamuel/Icog-Intern/blob/main/Prompt_Engineering/Advanced_Prompt_Engineering_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import random
import time

In [2]:
!pip install -q -U google-generativeai

In [4]:
!pip install dotenv

In [9]:
# library
import json
import os
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold
import time
from google.colab import userdata


In [12]:
# gemini api key
api_key = userdata.get('GOOGLE_API_KEY')

In [14]:
# llm model
genai.configure(api_key=api_key)
print("Gemini API Key configured successfully.")

Gemini API Key configured successfully.


In [15]:
# Select the Gemini model
# Other options: 'gemini-1.5-flash-latest', 'gemini-1.0-pro', etc.
ANALYSIS_MODEL_NAME = 'gemini-1.5-flash-latest'
RESPONSE_MODEL_NAME = 'gemini-1.5-flash-latest'

In [16]:
# Initialize the models
try:
    analysis_model = genai.GenerativeModel(ANALYSIS_MODEL_NAME)
    response_model = genai.GenerativeModel(RESPONSE_MODEL_NAME)
    print(f"Initialized Gemini models: {ANALYSIS_MODEL_NAME}, {RESPONSE_MODEL_NAME}")
except Exception as e:
    print(f"Error initializing Gemini models: {e}")
    exit()

Initialized Gemini models: gemini-1.5-flash-latest, gemini-1.5-flash-latest


In [25]:
# --- Updated LLM Call Function ---
def call_gemini_llm(prompt, model, model_name_for_log):
    """
    Calls the specified Gemini LLM model with the given prompt.
    Handles basic errors and safety blocking.
    """
    print(f"\n--- Sending Prompt to {model_name_for_log} ---")
    # print(prompt) # Uncomment to see the full prompt being sent
    print("--- Waiting for Gemini response... ---")

    try:
        # Use generate_content for non-streaming
        response = model.generate_content(
            prompt
            )

        print(f"--- Received Response from {model_name_for_log} ---")

        # Check for safety blocks or other issues before accessing text
        if not response.candidates:
             print("Warning: No candidates received from Gemini. Potential block or issue.")
             # Check prompt feedback if available
             try:
                 print(f"Prompt Feedback: {response.prompt_feedback}")
             except ValueError:
                 print("Prompt Feedback not available.")
             return None # Indicate failure

        # More detailed safety check (optional)
        first_candidate = response.candidates[0]
        if first_candidate.finish_reason != 1: # 1 typically means "STOP" (successful completion)
             print(f"Warning: Generation finished with reason: {first_candidate.finish_reason}")
             print(f"Safety Ratings: {first_candidate.safety_ratings}")
             # Consider returning None or handling based on finish_reason/safety_ratings

        # Attempt to get text, handle potential errors if part is missing
        try:
            response_text = response.text
            # print(response_text) # Uncomment to see raw response text
            print("--- Gemini Call Complete ---")
            return response_text
        except ValueError as e:
             # This can happen if the response was blocked entirely after generation started.
             print(f"Error extracting text from Gemini response: {e}")
             print(f"Prompt Feedback: {response.prompt_feedback}")
             print(f"Finish Reason: {first_candidate.finish_reason}")
             print(f"Safety Ratings: {first_candidate.safety_ratings}")
             return None # Indicate failure

    except Exception as e:
        print(f"Error during Gemini API call ({model_name_for_log}): {e}")
        return None # Indicate failure

In [26]:
# Technique 1: Chain-of-Thought (CoT) for Analysis
ANALYSIS_PROMPT_TEMPLATE = """
Analyze the following customer feedback. Provide your analysis *strictly* in JSON format with the keys "sentiment", "key_issues", and "reasoning".

Sentiment should be one of: "Positive", "Negative", "Neutral".
Key issues should be a list of specific topics or problems mentioned (even for positive feedback, list the positive aspects).
Reasoning should explain your step-by-step thinking process to arrive at the sentiment and key issues (Chain-of-Thought). Ensure the final output is only the valid JSON object.

Customer Feedback:
"{feedback_text}"

JSON Analysis:
"""

In [27]:
# Technique 2: Few-Shot Prompting for Response Generation
RESPONSE_PROMPT_TEMPLATE = """
You are a helpful and empathetic customer support agent for our company.
Draft a customer support response based on the original feedback and its analysis.
Keep the tone professional, empathetic, and constructive. Address the key issues identified. Be concise but thorough.

Here are some examples of good responses:

--- Example 1 ---
Feedback: The app is amazing! So easy to use and looks great.
Analysis: {{ "sentiment": "Positive", "key_issues": ["Ease of use", "Design"], "reasoning": "..." }}
Response: Thank you so much for your positive feedback! We're thrilled to hear you find the app easy to use and visually appealing. We appreciate you being a customer!

--- Example 2 ---
Feedback: Your checkout process is broken! I couldn't complete my purchase because the payment button didn't work. Very frustrating.
Analysis: {{ "sentiment": "Negative", "key_issues": ["Checkout process broken", "Payment button not working"], "reasoning": "..." }}
Response: We sincerely apologize for the frustration you experienced with our checkout process. We understand how important a smooth purchase experience is, and we're sorry the payment button wasn't working for you. Our technical team has been notified and is investigating this issue urgently. We appreciate your patience and for bringing this to our attention.

--- Example 3 ---
Feedback: The new feature is okay, but I think it could be improved by adding X.
Analysis: {{ "sentiment": "Neutral", "key_issues": ["New feature feedback", "Suggestion for improvement X"], "reasoning": "..." }}
Response: Thank you for sharing your thoughts on the new feature. We appreciate you taking the time to provide feedback and your suggestion about adding X. We'll pass this along to our product team for consideration as we continue to improve the feature based on user input.

--- Now, draft a response for the following ---

Original Feedback:
"{feedback_text}"

Analysis:
{analysis_json}

Draft Response:
"""


In [28]:
def analyze_feedback(feedback):
    """
    Analyzes customer feedback using the Gemini LLM with Chain-of-Thought prompting.
    """
    print("\n>>> Step 1: Analyzing Feedback...")
    prompt = ANALYSIS_PROMPT_TEMPLATE.format(feedback_text=feedback)

    raw_response = call_gemini_llm(prompt, analysis_model, f"{ANALYSIS_MODEL_NAME} (Analysis CoT)")

    if raw_response is None:
        print("Error: Failed to get analysis response from Gemini.")
        return None

    try:
        # Gemini might sometimes add ```json ... ``` markers, try to remove them
        if raw_response.strip().startswith("```json"):
             raw_response = raw_response.strip()[7:-3].strip()
        elif raw_response.strip().startswith("```"):
             raw_response = raw_response.strip()[3:-3].strip()

        analysis_result = json.loads(raw_response)

        # Basic validation
        if not all(k in analysis_result for k in ["sentiment", "key_issues", "reasoning"]):
            print("Error: Gemini analysis response is missing required keys.")
            print(f"Raw Response causing error: {raw_response}")
            return None # Failed validation

        print(">>> Analysis Complete:")
        print(f"   Sentiment: {analysis_result.get('sentiment')}")
        print(f"   Key Issues: {analysis_result.get('key_issues')}")
        # print(f"   Reasoning: {analysis_result.get('reasoning')}") # Optional
        return analysis_result

    except json.JSONDecodeError:
        print(f"Error: Failed to decode JSON response from Gemini analysis.")
        print(f"Raw Response: {raw_response}")
        return None # Failed JSON parsing
    except Exception as e:
        print(f"Error processing analysis response: {e}")
        return None


In [29]:
def generate_response(feedback, analysis_result):
    """
    Generates a draft response using the Gemini LLM with Few-Shot prompting.
    """
    print("\n>>> Step 2: Generating Response...")
    if not analysis_result:
        print("Error: Cannot generate response without valid analysis.")
        return None

    # Convert analysis dict back to JSON string for the prompt
    analysis_json = json.dumps(analysis_result, indent=2)

    prompt = RESPONSE_PROMPT_TEMPLATE.format(
        feedback_text=feedback,
        analysis_json=analysis_json
    )

    draft_response = call_gemini_llm(prompt, response_model, f"{RESPONSE_MODEL_NAME} (Response Few-Shot)")

    if draft_response is None:
        print("Error: Failed to get response draft from Gemini.")
        return None

    print(">>> Response Generation Complete:")
    # Clean up potential leading/trailing whitespace or markers if necessary
    draft_response_cleaned = draft_response.strip()
    print(f"   Draft Response: {draft_response_cleaned}")
    return draft_response_cleaned


In [37]:
def process_customer_feedback(customer_feedback):
    """
    Orchestrates the analysis and response generation process using Gemini.
    """
    print(f"\n===== Processing Feedback: '{customer_feedback}' =====")

    # Handle empty input edge case
    if not customer_feedback or not customer_feedback.strip():
        print("Error: Input feedback is empty.")
        return {"error": "Empty input feedback", "feedback": customer_feedback}

    # Step 1: Analyze Feedback (using CoT with Gemini)
    analysis = analyze_feedback(customer_feedback)

    if not analysis:
        print("Processing failed during analysis.")
        # Add delay before retrying if implementing retries
        # time.sleep(2)
        return {"error": "Analysis failed", "feedback": customer_feedback}

    # Add a small delay between API calls to potentially avoid rate limits
    time.sleep(1)

    # Step 2: Generate Response (using Few-Shot with Gemini)
    response = generate_response(customer_feedback, analysis)
    time.sleep(1)

    if not response:
        print("Processing failed during response generation.")
        # time.sleep(2) # Delay before potential retry
        return {"error": "Response generation failed", "feedback": customer_feedback, "analysis": analysis}

    print("\n===== Processing Complete =====")
    return {
        "original_feedback": customer_feedback,
        "analysis": analysis,
        "draft_response": response
    }

In [43]:
# --- Testing & Evaluation (Unchanged - uses the updated functions) ---

test_cases = [
    "The mobile app keeps crashing whenever I try to open the settings page. It's unusable!", # Negative, Bug
    "", # Edge Case: Empty String,
    "The website is generally fine, but loading times seem a bit slow sometimes, especially on the dashboard.", # Neutral/Mixed, Performance
    "Customer support was very unhelpful. They didn't understand my problem and closed the ticket.", # Negative, Support
    "Is there a way to export my data? I looked but couldn't find an option.", # Neutral, Question/Feature Request
    "Absolutely love the new dark mode feature! Makes using the app at night so much better. Thank you!", # Positive, Feature
    "asdfhjkl;", # Edge Case: Gibberish
    "This is okay I guess but the price is too high compared to competitors.", # Negative, Pricing
]

In [44]:
results = []
print("\n\n--- Starting Test Cases ---")
for i, feedback in enumerate(test_cases):
    print(f"\n--- Test Case {i+1} ---")
    result = process_customer_feedback(feedback)
    results.append(result)

    # Add a delay between processing test cases to respect potential API rate limits
    if "error" not in result: # Only delay if successful calls were made
         time.sleep(2) # Adjust delay as needed based on API limits (free tier often has strict limits)
    print("-" * 20)



--- Starting Test Cases ---

--- Test Case 1 ---

===== Processing Feedback: 'The mobile app keeps crashing whenever I try to open the settings page. It's unusable!' =====

>>> Step 1: Analyzing Feedback...

--- Sending Prompt to gemini-1.5-flash-latest (Analysis CoT) ---
--- Waiting for Gemini response... ---
--- Received Response from gemini-1.5-flash-latest (Analysis CoT) ---
--- Gemini Call Complete ---
>>> Analysis Complete:
   Sentiment: Negative
   Key Issues: ['Mobile app crashing', 'Settings page inaccessibility', 'App unusability']

>>> Step 2: Generating Response...

--- Sending Prompt to gemini-1.5-flash-latest (Response Few-Shot) ---
--- Waiting for Gemini response... ---
--- Received Response from gemini-1.5-flash-latest (Response Few-Shot) ---
--- Gemini Call Complete ---
>>> Response Generation Complete:
   Draft Response: We are so sorry to hear you're experiencing crashes when trying to access the settings page in our mobile app. We understand how frustrating and di

In [45]:
print("\n\n--- Test Results Summary ---")
for i, result in enumerate(results):
    print(f"\nTest Case {i+1}:")
    if "error" in result:
        print(f"  Status: FAILED ({result['error']})")
        print(f"  Input: '{result.get('feedback', test_cases[i])}'")
    else:
        print(f"  Status: SUCCESS")
        print(f"  Input: '{result['original_feedback']}'")
        print(f"  Sentiment: {result['analysis']['sentiment']}")
        print(f"  Key Issues: {result['analysis']['key_issues']}")
        # print(f"  CoT Reasoning: {result['analysis']['reasoning']}") # Can be verbose
        print(f"  Draft Response: {result['draft_response']}")




--- Test Results Summary ---

Test Case 1:
  Status: SUCCESS
  Input: 'The mobile app keeps crashing whenever I try to open the settings page. It's unusable!'
  Sentiment: Negative
  Key Issues: ['Mobile app crashing', 'Settings page inaccessibility', 'App unusability']
  Draft Response: We are so sorry to hear you're experiencing crashes when trying to access the settings page in our mobile app. We understand how frustrating and disruptive this must be, and we sincerely apologize for the inconvenience.  Our technical team is actively working to identify and resolve this issue.  Could you please provide us with your device's operating system and app version number so we can better investigate?  In the meantime, please know that we are prioritizing a fix to restore full functionality to the app. We appreciate your patience and understanding.

Test Case 2:
  Status: FAILED (Empty input feedback)
  Input: ''

Test Case 3:
  Status: SUCCESS
  Input: 'The website is generally fine, but lo